# Trabalho de Redes

## Importando bibliotecas:

In [2]:
pip install sklearn.model


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch 

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch 
import torch.nn as nn 
import torch.optim as optim
import optuna
from tqdm import tqdm

## REDE NEURAL

### Importando o Dataset

In [116]:
df = pd.read_csv("tiktok_treated_data.csv")
df_trabalho = pd.DataFrame(df)

### Separação dos dados:


In [117]:
df_trabalho.head(3)

,cosxt1,cosxt2,cosxt3,cosxt4,cosxt5,cosxt6,cosxt7,cosxt8,cosxt9,cosxt10,...,video_duration_sec,verified_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count,author_ban_status_active,author_ban_status_banned,author_ban_status_under review
0,-1.230874,-0.453840,0.032798,0.017766,0.026197,0.043926,0.021420,-0.177623,0.163330,0.350660,...,0.983333,0,0.343359,0.029529,0.000941,0.000067,0.000000,0.0,0.0,1.0
1,-0.908985,-0.135402,0.250411,-0.343244,-0.572894,1.521106,0.815831,0.221128,-0.500735,-0.999268,...,0.533333,0,0.140903,0.117591,0.074314,0.077431,0.071257,1.0,0.0,0.0
2,-1.341072,-0.467645,-0.117559,0.273214,-0.021553,-0.030836,0.197149,-0.380195,0.297951,-0.174708,...,0.516667,0,0.902350,0.148503,0.011158,0.055556,0.034274,1.0,0.0,0.0


In [118]:
# Todos os atributos da nossa rede com excessão do atributo target.
FEATURES = ['cosxt1', 'cosxt2', 'cosxt3', 'cosxt4', 'cosxt5', 'cosxt6', 'cosxt7',
       'cosxt8', 'cosxt9', 'cosxt10', 'claim_status', 'video_duration_sec',
       'verified_status', 'video_like_count',
       'video_share_count', 'video_download_count', 'video_comment_count',
       'author_ban_status_active', 'author_ban_status_banned',
       'author_ban_status_under review']

# Target da nossa Rede, no caso um single target. 
TARGET = ['video_view_count']

df_trabalho = df_trabalho.reindex(FEATURES + TARGET, axis=1)

In [120]:
TAMANHO_TESTE = 0.15
TAMANHO_VALIDACAO = 0.10
SEMENTE_ALEATORIA = 61455

# Aqui fazemos a divisão entre treino e teste. 
indices = df_trabalho.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Aqui fazemos a divisão do treinamento em treino e validação
indices_treino, indices_validacao = train_test_split(
    indices_treino, test_size=TAMANHO_VALIDACAO, random_state=SEMENTE_ALEATORIA
)

# Definindo os três dataset. 
df_treino = df_trabalho.loc[indices_treino]
df_validacao = df_trabalho.loc[indices_validacao]
df_teste = df_trabalho.loc[indices_teste]

# Dividindo entre o target e as features de cada dataset
X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()


X_validacao = df_validacao.reindex(FEATURES, axis=1).values
y_validacao = df_validacao.reindex(TARGET, axis=1).values.ravel()


X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()


# Covertendo os dados para tensores do pytorch
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_validacao = torch.tensor(X_validacao, dtype=torch.float32)
y_validacao = torch.tensor(y_validacao, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)


# Alterando as dimensões do target 
y_treino = y_treino.view(-1,1)
y_validacao = y_validacao.view(-1,1)
y_teste = y_teste.view(-1,1)

### Criando a Rede:

In [121]:
class view_predictor_MLP(nn.Module):
    def __init__(self, num_dados_entrada, estrutura, num_targets, funcao_ativacao, taxa_aprendizado, taxa_dropout):
        super(view_predictor_MLP, self).__init__()

        self.camadas =  nn.Sequential()
        
        # Definindo a função de ativação:
        if funcao_ativacao == "ReLU":
            self.ativacao = nn.ReLU()
        elif funcao_ativacao == "Sigmoid":
            self.ativacao = nn.Sigmoid()

        # Looping que adiciona as camadas dada uma lista, que contem o numero de camadas ocultas e neuronios. 
        for i in range(len(estrutura)):
            if i == 0:
                self.camadas.add_module('linear_{}'.format(i), nn.Linear(num_dados_entrada, estrutura[i]))
                self.camadas.add_module('sigmoid_{}'.format(i), self.ativacao)
            
            else:
                self.camadas.add_module('linear_{}'.format(i), nn.Linear(estrutura[i-1], estrutura[i]))
                self.camadas.add_module('sigmoid_{}'.format(i), self.ativacao)

        # adiciona a camadas final:
        self.camadas.add_module('linear_final', nn.Linear(estrutura[-1], num_targets))


        # Otimizador com Momento
        self.optimizer = optim.SGD(self.parameters(), lr=taxa_aprendizado)
        
        # Função de perda (Mean Squared Error)
        self.criterion = nn.MSELoss()

        # Define o Dropout
        self.dropout = nn.Dropout(p=taxa_dropout)
        
    def forward(self, x):

        # Condicional para quando estamos dentro do modo treinamento
        if self.training:
            for camada in self.camadas:
                
                 # Realiza o dropout na antes da aplicação da função linear, ou seja, antes de passar para a próxima camada a informação.
                if isinstance(camada, nn.Linear):
                    x = self.dropout(x)
                x = camada(x)

        # Se não estivermos no modo treinamento o dropout não é ativado. 
        else:
            for camada in self.camadas:
                x = camada(x)

        return x
    
    def treinamento(self, X_treino, y_treino, num_epocas):
        # Numero de treinamentos é definido pelo numero de epocas, ou seja, o numero de vezes que que nossa rede ve todos os dados de treino.
        
        for epoca in range(num_epocas):
            # Ativa o modo de treinamento
            self.train()
            
            # Realiza a propagação direta
            dados_de_saida = self(X_treino)
            
            # Calcula a perda:
            loss = self.criterion(dados_de_saida, y_treino)
            
            # Zera o gradiente:
            self.optimizer.zero_grad()

            # Realiza o Backpropagation:
            loss.backward()

            # Atualiza os parametros
            self.optimizer.step()
            

    def predicao(self, x):
        """
        Realiza a previsão a partir de um determinado conjunto de dados. 

        Args:
            X: Os dados do nosso dataset, varialvel no formato de tensor.

        Returns:
           retunr: Resultado das previsões com a rede, variável no formato de tensor. 
        """

        # Entra no modo previsão:
        self.eval()
        

        # Dertermina que não é mais necessario zerar o Gradiente.
        with torch.no_grad():
            dados_de_saida = self(x)
        

        return dados_de_saida
            

### Treinando a Rede Neural:


#### Otimização pelo Optuna:

In [127]:
# Aplicando nossa Rede Neural no Optura afim de obter os melhores hiperparametros. 

# Definindo algumas informações que seram usadas posteriormente.
NUM_DADOS_DE_ENTRADA = 20
NUM_DADOS_DE_SAIDA = 1
EPOCAS = 1000

# Definindo o número de arquiteturas passíveis de serem testadas.
num_arquitetura = 500

# Função objetivo para otimização 
def objetivo(trial):

    # Definindo os hiperparâmetros a serem otimizados
    num_camadas = trial.suggest_int('num_camadas', 2, 15)

    ESTRUTURA_OCULTA = []
    for i in range(num_camadas):
        num_neuronios = trial.suggest_int(f'num_neuronios_camada:{i}', 2, 50)
        ESTRUTURA_OCULTA.append(num_neuronios)
    
    FUNCAO_ATIVACAO = trial.suggest_categorical('funcao_de_ativacao', ['ReLU', 'Sigmoid'])
    TAXA_DE_APRENDIZADO = trial.suggest_float('taxa_de_aprendizado', 1e-5, 1e-1, log=True)
    TAXA_DE_DROPOUT = trial.suggest_float('taxa_de_dropout', 0.0, 0.5)


    # Criando uma instância do modelo com os hiperparâmetros sugeridos
    modelo = view_predictor_MLP(NUM_DADOS_DE_ENTRADA, ESTRUTURA_OCULTA, NUM_DADOS_DE_SAIDA, FUNCAO_ATIVACAO, TAXA_DE_APRENDIZADO, TAXA_DE_DROPOUT)
    
    # Treinando o modelo
    modelo.treinamento(X_treino, y_treino, EPOCAS)
    
    # Fazendo previsões no conjunto de validação
    y_predito = modelo.predicao(X_validacao)

    # Calcula o MSE no conjunto de validação
    mse = mean_squared_error(y_validacao, y_predito)
    
    return mse


# Executando a otimização do Optuna
study = optuna.create_study(direction='minimize')

# Cria uma  barra de progresso com um total igual ao número de arquiteturas a serem testadas
with tqdm(total=num_arquitetura) as pbar:
    # Define da função de atualização da barra de progresso
    def update_progress_bar(study, trial):
        pbar.update(1)  # Atualiza a barra de progresso em uma unidade a cada chamada
    
    # Executa a otimização com Optuna
    study.optimize(objetivo, n_trials=num_arquitetura, callbacks=[update_progress_bar])


# Obtendo os melhores hiperparâmetros encontrados
parametros = study.best_params

# Apartir do numero de neuronios por camada podemos montar a variável, ESTRUTURA_OCULTA
neuronios_camada = []
for parametro in parametros:
    if parametro.startswith('num_neuronios'):
        neuronios_camada.append(parametros[parametro])

[I 2024-05-13 18:59:19,641] A new study created in memory with name: no-name-b5de9f9e-02dc-40d6-9308-67b9b0c22768
100%|██████████| 500/500 [1:06:34<00:00,  7.99s/it]


#### Definindo os hiperparametros:

In [128]:
EPOCAS = 2000
NUM_DADOS_DE_ENTRADA = 20
ESTRUTURA_OCULTA = neuronios_camada
NUM_DADOS_DE_SAIDA = 1
FUNCAO_ATIVACAO = parametros['funcao_de_ativacao']
TAXA_DE_APRENDIZADO = parametros['taxa_de_aprendizado']
TAXA_DE_DROPOUT = parametros['taxa_de_dropout']

print('Hiperparametros da Rede:')
print(f'Numero de Dados de Entrada:{NUM_DADOS_DE_ENTRADA}')
print(f'Numero de Neuronios e Camadas: {ESTRUTURA_OCULTA}')
print(f'Função de ativação escolhida: {FUNCAO_ATIVACAO}')
print(f'Numero de Dados de Saída: {NUM_DADOS_DE_SAIDA}')
print(f'Taxa de apredizado do modelo: {TAXA_DE_APRENDIZADO}')
print(f'Taxa de Dropout do modelo: {TAXA_DE_DROPOUT}')

Hiperparametros da Rede:
Numero de Dados de Entrada:20
Numero de Neuronios e Camadas: [23, 30, 25, 31]
Função de ativação escolhida: ReLU
Numero de Dados de Saída: 1
Taxa de apredizado do modelo: 0.09966372836400053
Taxa de Dropout do modelo: 0.0004178587370442764


#### Treinando a rede com os melhores hiperparametros:

In [129]:

# Criando uma instância do modelo com os melhores hiperparâmetros
modelo = view_predictor_MLP(NUM_DADOS_DE_ENTRADA, ESTRUTURA_OCULTA, NUM_DADOS_DE_SAIDA, FUNCAO_ATIVACAO, TAXA_DE_APRENDIZADO, TAXA_DE_DROPOUT)

# Treinando o modelo
modelo.treinamento(X_treino, y_treino, EPOCAS)

### Testando a Rede Neural:

#### Prevendo os targets do X de tete:

In [130]:
# Realizando a previsão do numero de vizualizações com base no dataset X_teste
y_predito = modelo.predicao(X_teste)

#### Calculando a eficiencia da Previsão:

In [131]:
# Calculando a eficiência da Rede Neural, utilizando o Erro quadrático médio.
MSE = mean_squared_error(y_teste, y_predito)

#  Calculando a eficiência da Rede Neural, utilizando a Raiz do Erro quadrático médio.
RMSE = np.sqrt(MSE)

print(f' A rede possúi um MSE: {MSE} V² e um RMSE: {RMSE} V')

 A rede possúi um MSE: 0.02313041500747204 V² e um RMSE: 0.15208686888217926 V


# Referência:

[1] Video aulas e notebooks disponibilizados pelo professor Daniel Roberto Cassar.